### 场景

分析美国Texas旅游人数。数据集是每个城市游客人数的列表。现在要统计每个城市的游客占比。

In [5]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


### 需求增强 

为了适应从文件中读取大数据量的信息，于是定义生成器。

In [6]:
def read_visit(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [11]:
it = read_visit('my_data.txt')
percentages = normalize(it)
print(percentages)

[]


没有参数数据，原因是迭代器在sum语句中已经产生了一轮结果。在抛出StopIteration异常的**迭代器**或生成器上继续迭代第二轮，是不会有结果的。

In [12]:
it = read_visit('my_data.txt')
print(list(it))
print(list(it))

[15, 35, 80]
[]


由于上述异常没有报错，所以通常无法区分是迭代器没有数据，还是已经用完了。产生难以发现的bug。

### 解决方案一  --- list

In [14]:
def normalize_copy(numbers):
    numbers = list(numbers)
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

it = read_visit('my_data.txt')
percentages = normalize_copy(it)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这种方法与最初的方法一样没有解决内存消耗的问题

### 解决方案二 --- 函数参数lambda 

In [20]:
def normalize_func(get_iter):
    total = sum(get_iter())
    result = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
    return result

percentages = normalize_func(lambda: read_visit('my_data.txt'))
print(percentages)
#percentage = normalize_func(read_visit('my_data.txt'))
#print(percentage)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这种lambda的方式有点生硬。

### 解决方案三 --- 定义实现迭代器协议（iterator protocol）的容器类

**原理：**
for x in foo 语句会调用iter(foo)。内置的iter又会调用foo.\__iter\__方法。该方法（iter？）返回实现了名为\__next\__的迭代器（生成器）。此后，for循环会在迭代对象上反复调用内置的next函数，直至耗尽并产生StopIteration异常。

具体做法：通过添加\__itter\__方法实现生成器。最终的效果是把原来的生成器方法实现成了一个可迭代的容器类。

In [23]:
class ReadVisits(object):
    def __init__(self,data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

                
visits = ReadVisits('my_data.txt')
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


**分析：** normalize函数中的sum调用ReadVisits.\__iter\__,得到迭代器对象（生成器）。for语句也同样产生一个新的迭代器对象（生成器）。这两个迭代器互补干扰。

**小结：**
1. 把迭代器（生成器）传给iter函数，则返回迭代器（生成器）对象本身。
2. 把容器对象传给iter函数，则返回新的迭代器（生成器)对象。\__iter\__ 定义为生成器则返回生成器。

现在我们进一步修改最上面的normalize函数，判断参数是不是迭代器本身，如果是抛出TypeError错误。

In [25]:
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

现在该函数能够接受list和ReadVisit为输入参数，因为他们都是**容器**。凡是遵循迭代器协议的容器类型，都与这个函数相兼容。

In [27]:
visits = [15,35,80]
normalize_defensive(visits)
visits = ReadVisits('my_data.txt')
normalize_defensive(visits)

[11.538461538461538, 26.923076923076923, 61.53846153846154]

In [28]:
it = iter(visits)
normalize_defensive(it)

TypeError: Must supply a container

**要点：**
1. 函数在输入参数上进行多次迭代时需小心：如果参数是迭代器，可能导致奇怪结果或难以发现的bug。（改写成实现迭代器协议的容器类）
2. Python 迭代器协议，描述了容器和迭代器（生成器）如何与iter和next内置函数、for循环集相关表达式相互配合。
3. 把\__iter\__方法实现为生成器，即可定义成容器类型。
4. 判断某值是迭代器还是容器，可以拿该值作为参数，调用iter函数两次，若相同，则是迭代器。调用内置next函数，即可令迭代器前进一步。